In [1]:
!pip install evaluate

   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   --- ------------------------------------ 2.1/26.2 MB 9.0 MB/s eta 0:00:03
   ------ --------------------------------- 4.5/26.2 MB 9.9 MB/s eta 0:00:03
   --------- ------------------------------ 6.6/26.2 MB 10.3 MB/s eta 0:00:02
   ------------ --------------------------- 8.1/26.2 MB 10.3 MB/s eta 0:00:02
   --------------- ------------------------ 10.0/26.2 MB 9.3 MB/s eta 0:00:02
   ---------------- ----------------------- 11.0/26.2 MB 8.5 MB/s eta 0:00:02
   ------------------ --------------------- 12.1/26.2 MB 8.2 MB/s eta 0:00:02
   -------------------- ------------------- 13.4/26.2 MB 7.8 MB/s eta 0:00:02
   ---------------------- ----------------- 14.7/26.2 MB 7.6 MB/s eta 0:00:02
   ------------------------ --------------- 16.0/26.2 MB 7.5 MB/s eta 0:00:02
   -------------------------- ------------- 17.3/26.2 MB 7.4 MB/s eta 0:00:02
   ---------------------------- ----------- 18.9/26.2 MB 7.4 MB/s eta 0:00

In [12]:
import pandas as pd
import os

# Yüklenen veri setinin yolu
csv_file_path = "e-ticaret_urun_yorumlari.csv" # Dosya yolunu düzelttik

# CSV dosyasını pandas DataFrame'e yükleyelim
try:
    # Ayırıcıyı noktalı virgül (';') olarak belirtelim ve hatalı satırları atlayalım
    df_reviews = pd.read_csv(csv_file_path, sep=';', on_bad_lines='skip')


    # İlk birkaç satırını görüntüleyelim
    print("Veri Seti (İlk 5 Satır):")
    display(df_reviews.head())

    # Veri seti hakkında genel bilgi alalım (sütunlar, veri tipleri, eksik değerler)
    print("\nVeri Seti Bilgisi:")
    df_reviews.info()

    # Sütun adlarını kontrol edelim (Bilindiği için doğrudan kullanacağız)
    text_column_name = 'Metin'
    label_column_name = 'Durum'
    print(f"\nMetin Sütunu Adı: {text_column_name}")
    print(f"Etiket Sütunu Adı: {label_column_name}")

    # Etiket dağılımına bakalım
    if label_column_name in df_reviews.columns:
        print(f"\n'{label_column_name}' Sütunu Dağılımı:")
        print(df_reviews[label_column_name].value_counts())
        # Burada num_labels ve label_map'i fine-tuning adımı için saklayabiliriz
        unique_labels = df_reviews[label_column_name].unique()
        # np.int64 objelerini int'e dönüştürelim
        label_map = {int(label): i for i, label in enumerate(unique_labels)}
        num_labels = len(unique_labels)
        print(f"\nEtiket Eşlemesi (Fine-tuning için): {label_map}")
        print(f"Toplam Etiket Sayısı (Fine-tuning için): {num_labels}")

    else:
        # Bu durum normalde olmamalı çünkü sütun adlarını doğrudan kullandık,
        # ama hata kontrolü için burada bırakılabilir veya kaldırılabilir.
        print(f"\nHata: '{label_column_name}' sütunu veri setinde bulunamadı.")


except FileNotFoundError:
    print(f"Hata: {csv_file_path} bulunamadı. Lütfen dosya yolunu kontrol edin.")
except Exception as e:
    print(f"Veri seti yüklenirken bir hata oluştu: {e}")

Veri Seti (İlk 5 Satır):


,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1



Veri Seti Bilgisi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15170 entries, 0 to 15169
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Metin   15170 non-null  object
 1   Durum   15170 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 237.2+ KB

Metin Sütunu Adı: Metin
Etiket Sütunu Adı: Durum

'Durum' Sütunu Dağılımı:
Durum
0    6978
1    6799
2    1393
Name: count, dtype: int64

Etiket Eşlemesi (Fine-tuning için): {1: 0, 0: 1, 2: 2}
Toplam Etiket Sayısı (Fine-tuning için): 3


In [14]:
from transformers import AutoTokenizer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import os

# Yüklenen veri setinin yolu (bir önceki adımdan)
csv_file_path = "e-ticaret_urun_yorumlari.csv"

# CSV dosyasını pandas DataFrame'e yükleyelim (bir önceki adımdan)
try:
    # Ayırıcıyı noktalı virgül (';') olarak belirtelim ve hatalı satırları atlayalım
    df_reviews = pd.read_csv(csv_file_path, sep=';', on_bad_lines='skip')
    print("Veri seti başarıyla yüklendi.")
except FileNotFoundError:
    print(f"Hata: {csv_file_path} bulunamadı. Lütfen dosya yolunu kontrol edin.")
    df_reviews = None
except Exception as e:
    print(f"Veri seti yüklenirken bir hata oluştu: {e}")
    df_reviews = None


# df_reviews'ın yüklenip yüklenmediğini kontrol edelim
if df_reviews is not None and not df_reviews.empty:
    # Sütun adları 'Metin' ve 'Durum' olarak bilindiği için doğrudan kullanalım.
    text_column = 'Metin' # Metin sütununun adı
    label_column = 'Durum' # Etiket/Duygu sütununun adı

    # Sadece ilgili sütunları seçelim ve eksik değerleri temizleyelim (varsa)
    df_cleaned = df_reviews[[text_column, label_column]].dropna().copy()

    # Etiket sütununun adını 'labels' olarak değiştirelim (Hugging Face Trainer beklentisine uygun)
    df_cleaned = df_cleaned.rename(columns={label_column: 'labels'})

    # Etiketleri sayısal değerlere dönüştürelim ve label_map ile num_labels'ı tanımlayalım
    # Etiketler zaten sayısal (int64) olduğu için doğrudan unique değerleri alabiliriz.
    unique_labels = df_cleaned['labels'].unique()
    # np.int64 objelerini int'e dönüştürelim ki JSON serileştirme veya diğer kullanımlar sorun yaratmasın
    label_map = {int(label): i for i, label in enumerate(unique_labels)}
    num_labels = len(unique_labels)


    print(f"Etiket Eşlemesi: {label_map}")
    print(f"Toplam Etiket Sayısı: {num_labels}")

    # Eğitim ve test setlerine ayıralım
    # Stratify kullanarak etiket dağılımını eğitim ve test setlerinde benzer tutalım
    train_df, test_df = train_test_split(df_cleaned, test_size=0.2, random_state=42, stratify=df_cleaned['labels'])

    # Hugging Face Datasets formatına dönüştürelim
    # 'Metin' sütununu 'text' olarak yeniden adlandıralım ki tokenizer kolayca işleyebilsin
    train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True)).rename_column("Metin", "text")
    test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True)).rename_column("Metin", "text")


    # Bir belirteçleyici yükleyelim
    # Türkçe için uygun bir model seçelim.
    model_checkpoint = "dbmdz/bert-base-turkish-cased"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    # Veriyi belirteçlere ayıralım
    def tokenize_function(examples):
        # Tokenizer'ın beklediği formatta metin sütununu kullanalım
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128) # max_length ayarlanabilir

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


    # Gereksiz sütunları kaldıralım (original text column)
    tokenized_train_dataset = tokenized_train_dataset.remove_columns(['text']) # 'text' sütununu kaldırdık
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(['text']) # 'text' sütununu kaldırdık


    # Formatı ayarlayalım
    tokenized_train_dataset.set_format("torch")
    tokenized_test_dataset.set_format("torch")


    print("\nTokenize Edilmiş Eğitim Veri Seti:")
    print(tokenized_train_dataset)
    print("\nTokenize Edilmiş Test Veri Seti:")
    print(tokenized_test_dataset)

    # num_labels ve label_map'i de yazdıralım
    print(f"\nEtiket Eşlemesi (Fine-tuning için): {label_map}")
    print(f"Toplam Etiket Sayısı (Fine-tuning için): {num_labels}")

else:
    print("DataFrame yüklenemediği veya boş olduğu için sonraki adımlar atlandı.")

Veri seti başarıyla yüklendi.
Etiket Eşlemesi: {1: 0, 0: 1, 2: 2}
Toplam Etiket Sayısı: 3


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /dbmdz/bert-base-turkish-cased/resolve/main/tokenizer_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))"), '(Request ID: 34f455eb-4ffc-47d2-9ad9-09d7a85a37aa)')

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

# Sınıf sayısını belirleyelim (veri hazırlığı adımından gelen num_labels değişkeni)
# Eğer önceki hücreyi çalıştırmadıysanız veya değişken kaybolduysa buraya manuel olarak atayabilirsiniz.
# num_labels = 3 # Örnek değer, veri setinizdeki etiket sayısına göre ayarlayın

# Modeli yükleyelim
# model_checkpoint değişkeni veri hazırlığı adımından geliyor olmalı.
# Eğer gelmiyorsa buraya manuel olarak atayın: model_checkpoint = "dbmdz/bert-base-turkish-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# Metrik fonksiyonunu tanımlayalım (doğruluk)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Eğitim argümanlarını tanımlayalım
training_args = TrainingArguments(
    output_dir="./results",          # Çıktı dizini
    learning_rate=2e-5,              # Öğrenme oranı
    per_device_train_batch_size=16,  # Cihaz başına eğitim toplu iş boyutu (belleğe göre ayarlayın)
    per_device_eval_batch_size=16,   # Cihaz başına değerlendirme toplu iş boyutu (belleğe göre ayarlayın)
    num_train_epochs=3,              # Toplam eğitim epoch sayısı (küçük veri seti için yeterli olabilir)
    weight_decay=0.01,               # Ağırlık düşürme
    eval_strategy="epoch",           # Her epoch sonunda değerlendir
    save_strategy="epoch",           # Her epoch sonunda kaydet
    load_best_model_at_end=True,     # Eğitim sonunda en iyi modeli yükle
    report_to="none",                # Weights & Biases entegrasyonunu kapat
    push_to_hub=False,               # Modeli Hugging Face Hub'a yükleme
)

# Trainer sınıfını oluşturalım
# tokenizer değişkeni veri hazırlığı adımından geliyor olmalı.
trainer = Trainer(
    model=model,                         # Eğitilecek model
    args=training_args,                  # Eğitim argümanları
    train_dataset=tokenized_train_dataset, # Eğitim veri seti
    eval_dataset=tokenized_test_dataset,   # Değerlendirme veri seti
    tokenizer=tokenizer,                 # Belirteçleyici
    compute_metrics=compute_metrics      # Metrik fonksiyonu
)

# Modeli fine-tune edelim
print("Model fine-tuning ediliyor...")
trainer.train()
print("Fine-tuning tamamlandı.")

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /dbmdz/bert-base-turkish-cased/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))"), '(Request ID: 49cda4c5-1f95-4520-971f-a095cc46f451)')

In [16]:
# Modeli test veri seti üzerinde değerlendirelim
print("Model değerlendiriliyor...")
evaluation_results = trainer.evaluate()

# Değerlendirme sonuçlarını yazdıralım
print("\nDeğerlendirme Sonuçları:")
print(evaluation_results)

# Özellikle doğruluk (accuracy) gibi metrikleri kontrol edelim
if 'eval_accuracy' in evaluation_results:
    print(f"\nTest Seti Doğruluğu: {evaluation_results['eval_accuracy']:.4f}")

Model değerlendiriliyor...


NameError: name 'trainer' is not defined

In [17]:
# sentence-transformers kütüphanesini kuralım (eğer daha önce kurulmadıysa)
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np
import torch # PyTorch'u içe aktaralım

# Kullanılacak embedding modelini yükleyelim.
# Türkçe için uygun bir model seçmek önemlidir.
# Örneğin: 'paraphrase-multilingual-MiniLM-L12-v2' veya 'sentence-transformers/stsb-bert-base-multilingual-uncased'
# Büyük bir veri setiyle çalışıyorsanız, daha küçük bir model seçmek kaynak kullanımı açısından daha iyi olabilir.
# Örnek için 'paraphrase-multilingual-MiniLM-L12-v2' kullanalım.
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Bilgi kaynağımızın embeddinglerini oluşturalım
# knowledge_source değişkeni bir önceki adımdan geliyor olmalı.
if 'knowledge_source' in locals() and len(knowledge_source) > 0:
    print(f"{len(knowledge_source)} adet yorumun embeddingleri oluşturuluyor...")

    # Bellek sorunlarını önlemek için büyük veri setlerini batch'ler halinde işlemek iyi bir fikirdir.
    # Ancak bu örnek için doğrudan tüm yorumları işleyelim.
    # Eğer bellek hatası alırsanız, burayı batch işlemeye göre yeniden düzenlemeniz gerekebilir.
    knowledge_embeddings = embedding_model.encode(knowledge_source, show_progress_bar=True)

    print("Embedding oluşturma tamamlandı.")
    print(f"Oluşturulan embeddinglerin şekli: {knowledge_embeddings.shape}") # (yorum sayısı, embedding boyutu)

    # Embeddingleri ve bilgi kaynağını daha sonra kullanmak üzere saklayalım
    # knowledge_embeddings ve knowledge_source artık RAG sistemimizin temel bileşenleri.

else:
    print("Bilgi kaynağı (knowledge_source) bulunamadı veya boş. Lütfen bir önceki adımı çalıştırın.")

No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/adapter_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))"), '(Request ID: 08495732-fbd7-417d-9bd9-f7baccc4d97b)')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Kullanıcı sorgusunu tanımlayalım
user_query = "Ürün kalitesi nasıl?"

# Kullanıcı sorgusunun embeddingini oluşturalım
# knowledge_embeddings oluşturmak için kullandığımız aynı modeli kullanalım
if 'embedding_model' in locals():
    query_embedding = embedding_model.encode([user_query])[0] # Sorgu için embedding oluştur

    # Sorgu embeddingi ile bilgi kaynağı embeddingleri arasındaki benzerliği (kosinüs benzerliği) hesaplayalım
    # knowledge_embeddings değişkeni bir önceki adımdan geliyor olmalı.
    if 'knowledge_embeddings' in locals() and 'knowledge_source' in locals():
        print(f"'{user_query}' sorgusu için ilgili yorumlar aranıyor...")

        # Kosinüs benzerliğini hesapla (vektörlerin normalleştirildiğini varsayarak dot product da kullanılabilir)
        # Embeddingler zaten SentenceTransformer tarafından normalleştirilmiş olabilir.
        # Güvenli olmak için cosine_similarity kullanalım.
        similarities = cosine_similarity([query_embedding], knowledge_embeddings)[0]

        # En yüksek benzerliğe sahip yorumların indekslerini alalım
        # Örneğin, en ilgili 5 yorumu bulalım
        top_k = 5
        top_k_indices = np.argsort(similarities)[::-1][:top_k]

        print(f"\n'{user_query}' sorgusu ile En Benzer (En Alakalı) {top_k} Yorum:")
        retrieved_reviews = [knowledge_source[i] for i in top_k_indices]

        for i, review in enumerate(retrieved_reviews):
            print(f"{i+1}. Benzerlik: {similarities[top_k_indices[i]]:.4f} - Yorum: \"{review}\"")

        # --- Cevap Üretme Simülasyonu ---
        # Gerçek RAG sisteminde, bu retrieved_reviews bir dil modeline prompt olarak verilirdi.
        # Örneğin: "Aşağıdaki ürün yorumlarını kullanarak 'Ürün kalitesi nasıl?' sorusuna kısa bir cevap ver:\n\n" + "\n".join(retrieved_reviews)
        # Burada basitçe ilgili yorumları bir araya getirerek bir cevap simülasyonu yapalım.

        print("\n--- RAG Tabanlı Cevap Simülasyonu ---")
        simulated_answer = "Kullanıcı yorumlarına göre ürün kalitesi hakkında bazı bilgiler:\n\n" + "\n".join([f"- {review}" for review in retrieved_reviews])

        print(simulated_answer)
        print("-" * 30)
        print("Not: Bu, ilgili yorumları temel alan basit bir simülasyondur. Gerçek bir RAG modeli daha akıcı ve bağlamsal cevaplar üretecektir.")


    else:
        print("Bilgi kaynağı embeddingleri (knowledge_embeddings) veya bilgi kaynağı (knowledge_source) bulunamadı. Lütfen önceki adımları çalıştırın.")

else:
    print("Embedding modeli (embedding_model) bulunamadı. Lütfen önceki adımları çalıştırın.")

In [ ]:
# Örnek bir yorum seçelim (test setinden veya bilgi kaynağından olabilir)
# Örnek olarak RAG sorgusuna benzer bir yorum alalım:
sample_review = "Ürünün hem fiyatı uygun hemde kalitesi çok iyi."

# Etiket eşlemesini de kullanalım (veri hazırlığı adımından)
# Eğer bu hücreyi tek başına çalıştırıyorsanız, label_map'i yeniden tanımlamanız gerekebilir.
# label_map = {1: 0, 0: 1, 2: 2} # Örnek değer, kendi label_map'inizle değiştirin
# Ters eşlemeyi oluşturalım: id_to_label = {v: k for k, v in label_map.items()}
id_to_label = {0: 1, 1: 0, 2: 2} # Modelin çıktısı (0, 1, 2) -> Orijinal etiket (1, 0, 2)


# Yorumu tokenize edelim
inputs = tokenizer(sample_review, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Modeli ve girdi tensörlerini aynı cihaza taşıyalım (GPU varsa 'cuda', yoksa 'cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}


# Tahmin yapalım
model.eval() # Modeli değerlendirme moduna alalım
with torch.no_grad(): # Gradyan hesaplamayı kapat
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    predicted_label_id = predictions.item()
    predicted_sentiment = id_to_label[predicted_label_id]


print(f"Örnek Yorum: \"{sample_review}\"")
print(f"Fine-tuning Modeli Duygu Tahmini (Sayısal Etiket): {predicted_label_id}")
# Etiket eşlemesini kullanarak anlamlı etiketi yazdıralım
print(f"Fine-tuning Modeli Duygu Tahmini (Orijinal Etiket): {predicted_sentiment}")


print("\n--- Karşılaştırma ---")

# Daha önceki RAG simülasyonu çıktısını hatırlayalım
# user_query ve simulated_answer değişkenlerinin önceki hücrelerden geldiğini varsayıyoruz.
if 'user_query' in locals() and 'simulated_answer' in locals():
    print(f"Daha önceki RAG Sorgusu: '{user_query}'")
    print("RAG Simülasyonu Çıktısı (İlgili Yorumları Getirme ve Özetleme):")
    print(simulated_answer) # RAG simülasyonundan gelen değişken
else:
    print("RAG simülasyonu değişkenleri (user_query, simulated_answer) bulunamadı. Lütfen RAG simülasyonu adımını çalıştırın.")


print("\nFark:")
print("Fine-tuning modeli, doğrudan yorumun duygu sınıfını (pozitif, negatif, nötr vb.) tahmin etti.")
print("RAG simülasyonu ise, sorguyla ilgili yorumları bilgi kaynağından getirdi ve bu yorumları kullanarak bir 'cevap' simüle etti.")
print("Fine-tuning metnin genel duygu tonunu anlamaya odaklanırken, RAG belirli bir soruya harici bilgiden faydalanarak yanıt bulmaya odaklanır.")